In [2]:
import pandas as pd
import numpy as np

# Data Preparation

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv")

In [4]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [5]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [6]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [7]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [8]:
categorical = df.select_dtypes(include=['object', 'string']).columns.tolist()
categorical

['lead_source', 'industry', 'employment_status', 'location']

In [9]:
numerical = ["number_of_courses_viewed", "annual_income", "interaction_count", "lead_score"]

In [10]:
df[categorical].isnull().sum()

lead_source          128
industry             134
employment_status    100
location              63
dtype: int64

In [11]:
df[categorical] = df[categorical].fillna("NA")

In [12]:
df[categorical].isnull().sum()

lead_source          0
industry             0
employment_status    0
location             0
dtype: int64

In [13]:
df[numerical].isnull().sum()

number_of_courses_viewed      0
annual_income               181
interaction_count             0
lead_score                    0
dtype: int64

In [14]:
df[numerical] = df[numerical].fillna(0.0)

In [15]:
df[numerical].isnull().sum()

number_of_courses_viewed    0
annual_income               0
interaction_count           0
lead_score                  0
dtype: int64

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_auc_score

In [23]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [24]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [25]:
df.shape

(1462, 9)

# Question 1

In [26]:
numerical = ["number_of_courses_viewed", "annual_income", "interaction_count", "lead_score"]

In [31]:
y_train = df_train["converted"].values

In [32]:
del df_train["converted"]

In [33]:
df_train

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
462,events,manufacturing,2,95543.0,unemployed,europe,3,0.78
135,referral,NA,1,54924.0,student,south_america,6,0.39
117,organic_search,healthcare,2,77352.0,unemployed,europe,2,0.22
1051,paid_ads,other,2,34600.0,employed,south_america,2,0.31
945,paid_ads,education,0,43615.0,unemployed,south_america,2,0.01
...,...,...,...,...,...,...,...,...
39,NA,other,5,67314.0,NA,europe,2,0.87
1261,events,education,6,63996.0,NA,australia,4,0.92
595,organic_search,finance,1,73702.0,unemployed,north_america,2,0.55
1329,events,technology,1,93341.0,student,middle_east,4,0.99


In [34]:
roc_values = {}

for col in numerical:
    try:
        auc = roc_auc_score(y_train, df_train[col])
        roc_values[col] = auc
    except ValueError:
        roc_values[col] = None

roc_sorted = pd.Series(roc_values).sort_values(ascending=False)
print(roc_sorted)

number_of_courses_viewed    0.763568
interaction_count           0.738270
lead_score                  0.614499
annual_income               0.551958
dtype: float64
